## Training the fine-tuned VGG16 model with _automatic mixed precision training_

This notebook shows the runtime improvement happened due to the mixed precision training and _the score also got improved_. To know more about mixed precision training, check out the following resources - 

* https://www.tensorflow.org/api_docs/python/tf/contrib/mixed_precision/LossScaleOptimizer
* https://docs.nvidia.com/deeplearning/sdk/mixed-precision-training/index.html#example_tensorflow

With `TensorFlow 1.13`, it is possible to train a neural network (within a NVIDIA GPU-enabled environment) just adding the following line of code. 

**Note** that, I used Google Cloud Platform for this purpose and used a standard [n1-standard-4 machine](https://cloud.google.com/compute/docs/machine-types) with **Tesla P4** GPUs. 

In [1]:
import os
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'

In [2]:
import pandas as pd
import numpy as np
import random
from scipy.misc import imresize
from scipy.misc import imshow
from scipy.misc import imread
import os
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
%matplotlib inline

# For fixing the randomness and thereby enabling my team members to reproduce the experiments
np.random.seed(7)

import warnings
warnings.filterwarnings("ignore")

In [3]:
import tensorflow
from tensorflow.keras.applications.vgg16 import VGG16

In [7]:
!pwd

/home/jupyter/histology_classifier/Sayak's Notebooks


In [9]:
!ls /home/jupyter/histology_classifier/

Anirudh's Notebooks  Dr. Sanyal's Basic Notebook  README.md
data_chec_resized    __pycache__		  Sayak's Notebooks


In [4]:
filename_csv = pd.read_csv('/home/jupyter/histology_classifier/data_chec_resized/dat.csv', sep=',')
temp = []
for img_name in filename_csv.filename:
    img_path = os.path.join('/home/jupyter/histology_classifier/data_chec_resized', 'pool', img_name)
    img = imread(img_path)
    img = imresize(img, (256,192))
    #img = img.astype('float32') 
    temp.append(img)

X = np.stack(temp)

In [5]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()

y = lb.fit_transform(filename_csv['label'])
y = tensorflow.keras.utils.to_categorical(y)

In [6]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [7]:
base_model = VGG16(weights='imagenet', 
                       include_top=False, 
                       input_shape=(256, 192, 3))

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
# Extract the last layer from third block of vgg16 model
last = base_model.get_layer('block3_pool').output

In [9]:
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization, Dense, Dropout
from tensorflow.keras.models import Model

In [10]:
# Add classification layers on top of it
x = GlobalAveragePooling2D()(last)
x= BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.6)(x)
pred = Dense(8, activation='softmax')(x)
model = Model(base_model.input, pred)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 192, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 192, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 192, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 96, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 96, 128)      73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 96, 128)      147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 48, 128)       0         
__________

In [12]:
from tensorflow.keras import optimizers

for layer in base_model.layers:
     layer.trainable = False

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-4),
              metrics=['accuracy'])

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

trainAug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.05,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    horizontal_flip=True,
    fill_mode="nearest")
 

valAug = ImageDataGenerator()

In [14]:
trainAug.fit(X_train)
train_generator = trainAug.flow(X_train, y_train, batch_size=32)
                                     
valAug.fit(X_test)
valid_generator = valAug.flow(X_test, y_test, batch_size=32)

In [15]:
%%timeit
history = model.fit_generator(train_generator,
                              steps_per_epoch=2,
                              validation_data=valid_generator,
                              validation_steps=2,
                              epochs=64,
                              verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/64
2/2 [==============================] - 10s 5s/step - loss: 2.6373 - acc: 0.0938 - val_loss: 9.8712 - val_acc: 0.1875
Epoch 2/64
2/2 [==============================] - 0s 235ms/step - loss: 2.5895 - acc: 0.1875 - val_loss: 8.4953 - val_acc: 0.1875
Epoch 3/64
2/2 [==============================] - 1s 361ms/step - loss: 2.4437 - acc: 0.1406 - val_loss: 7.7955 - val_acc: 0.1875
Epoch 4/64
2/2 [==============================] - 1s 321ms/step - loss: 2.1265 - acc: 0.2031 - val_loss: 7.3794 - val_acc: 0.1875
Epoch 5/64
2/2 [==============================] - 1s 361ms/step - loss: 1.9876 - acc: 0.2656 - val_loss: 6.9330 - val_acc: 0.1875
Epoch 6/64
2/2 [==============================] - 1s 308ms/step - loss: 2.0104 - acc: 0.1875 - val_loss: 6.5128 - val_acc: 0.1250
Epoch 7/64
2/2 [==============================] - 1s 346ms/step - loss: 1.9507 - acc: 0.2344 - val_loss: 5.9950 - val_acc: 0.1250
Epoch 8/64
2/2 [============================

The main highlight is the following (runtime): 
> **43.3 s ± 824 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)**

Which is **10x faster** than the previous one. 

In [18]:
preds_vgg = model.predict(X_test)
preds_vgg.shape

(16, 8)

In [19]:
print(accuracy_score(np.argmax(y_test, axis=1), np.argmax(preds_vgg, axis=1))*100.)

87.5


Improved score as well. 

In [20]:
model.save_weights('model_weights/vgg_fine_tuned_with_mixed_precision.h5')